In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import constants as C
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
from astropy import units as u

**Conditional about observations:**

The user gives the latitude and the functions returns if the phenomenon can be see it or not. This using the conditional about declination and latitude.

Alert system's format DESI:


In [ ]:
lat_obs = 4  # Latitud del observatorio en grados 
lon_obs = 74  # Longitud del observatorio en grados 
ra_list = [83.633, 85.792, 82.985]  # Lista de RA en grados
dec_list = [80.014, 78.091, 50.568]  # Lista de Dec en grados

tiempo = Time('2025-02-13 22:00:00')

observatorio = EarthLocation(lat=lat_obs*u.deg, lon=lon_obs*u.deg)

coordenadas_celestes = SkyCoord(ra=ra_list*u.deg, dec=dec_list*u.deg)

    # Calcular las coordenadas altazimutales para el tiempo y ubicación dados
frame_altaz = AltAz(obstime=tiempo, location=observatorio)
coordenadas_altaz = coordenadas_celestes.transform_to(frame_altaz)
    
    # Determinar si el objeto es observable (altitud > 0 grados)
observable = coordenadas_altaz.alt > 0*u.deg

coordenadas_altaz

<SkyCoord (AltAz: obstime=2025-02-13 22:00:00.000, location=(1753798.85431485, 6116223.45612925, 441945.105248) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    [(350.30593765,  1.64043292), (348.33998787,  1.5837905 ),
     (321.85806626, -6.13559436)]>

In [25]:
#c.to_string('hmsdms') '00h42m44.2992s +41d16m09.012s'
#c.to_string('decimal') '10.56 +29.865'
a = SkyCoord(ra='00h42m44.2992s', dec='+41d16m09.012s')
a.transform_to(frame_altaz) 


<SkyCoord (AltAz: obstime=2025-02-13 22:00:00.000, location=(1753798.85431485, 6116223.45612925, 441945.105248) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (357.01634897, -44.51787807)>

Retornar las coordenadas dle objeto con la lista de booleanos y la grafica de la carta celeste